# Let's Build Personalized Website using Agentic Workflows and Inferentia

Notes: 
1. Use the native Jupyter Notebook to leverage the full features of this notebook
2. Ensure you have Bedrock connection with access to Titan Models 

## 1. Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import base64
import boto3
import io
import os
import json
import re
import importlib
import shutil
import re
import numpy as np
import matplotlib.pyplot as plt

from src import Bedrock

from ipywidgets import widgets
from IPython.core.display import display, HTML
from PIL import Image


In [ ]:
importlib.reload(Bedrock)

### Useful functions

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    total_text = ""
    for result in retrievalResults.get("retrievalResults", []):
        content = result.get("content", {})
        text = content.get("text", "")
        # Remove the metadataAttributes content from the text
        text = re.sub(r'\{.*?metadataAttributes.*?\}', '', text)
        total_text += text
    return total_text

In [ ]:
def build_profile(UserProfile):
    profile = f"Your customer is {UserProfile['Name']}, located in {UserProfile['Location']}. They are a {UserProfile['Industry']} company. Their mission statement is `{UserProfile['Mission']}."
    return profile

### Declerations

In [ ]:
# create general model inputs 
# do not edit any content in this section

#image_modelId = 'amazon.titan-image-generator-v1'
#model_id = {'Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
#           'Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
#           'StableDiff': 'stability.stable-diffusion-xl-v1'}

In [ ]:
# ENV VARIABLES
kbId = '2TBRS1XZBR'
model_llama3p1_endpoint = 'nous-research-Meta-Llama-3-8B-2024-09-23-19-42-59-162-endpoint'

### Checking the endpoint status

In [ ]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_llama3p1_endpoint)
print("Status: " + resp["EndpointStatus"])

### INPUT

Offerings

In [ ]:
with open("./references/offerings.json", "r") as file:
    offerings = json.load(file)

User Profiles

In [ ]:
Profiles = {
    'Construction-Example': {
        'Name': 'Example Corp Construction Inc',
        'Industry': 'Construction',
        'Location': 'New York City, NY',
        'Mission': 'Building a sustainable future for New York'
    },
    'Manufacturing-Example': {
        'Name': 'Example Corp Manuf LLC',
        'Industry': 'Manufacturing',
        'Location': 'San Jose, CA',
        'Mission': 'Building the next generation Electric Vehicles'
    },
    'Mining-Example': {
        'Name': 'Example Corp Mining Inc',
        'Industry': 'Mining',
        'Location': 'Bisbee, AZ',
        'Mission': 'Extracting the value for America'
    }
}

Now a user signs in (ITERATION)

In [ ]:
# ITERATION - SELECTION:
selected_profile = 'Manufacturing-Example'

In [ ]:
UserProfile= Profiles[selected_profile]
offering = re.sub(r'\n#?', ' ', offerings[customer['Industry']])
project = selected_profile + "_Website_Test"

### Runtime

#### 1) Natural Language description of customer profile

In [ ]:
customer = UserProfile.copy()
customer['Description'] = build_profile(UserProfile)
print(customer['Description'])

#### 2) Retreieve painpoints of customer industry / any context
This flow uses RAG to retreive documents

In [ ]:
query = f"{customer['Description']}\nList the industry pain-points and challenges"
print(f"Here is your customer:\n{query}")

In [ ]:
# Let's find the context for the painpoints and challenges for this customer using RAG.
context_painpoints = Bedrock.dec_retrieve(query, kbId, numberOfResults=5)
contexts_painpoints = get_contexts(context_painpoints)
contexts_painpoints = re.sub(r'\n#?', ' ', contexts_painpoints)
print(f"These are the applicable pain-points/challenges:\n{contexts_painpoints}")

### 3) Prompting AI to describe a website personalized for this profile and background
Here the prompting is critical. Please see the guidance provided to the LLM. We also ask the response to be in two parts; 1/ Description, 2/ Visual Elements. The second part is needed for an easier traffic move to image generation or finding image.

#### 3.1) Finding the possible painpoints using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant in AnyCompany Consulting.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** AnyCompany Consulting Research Document ***
{contexts_painpoints}
*** End of AnyCompany Consulting Research Document ***


Question: {customer['Description']} Based on the above research, list top-3 challenges for {customer['Name']}. Provide your list in bullet points without any preamble (e.g. • a pain-point, • another pain-point, etc.). 

"""
params = {"max_new_tokens": 256, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_painpoints = json.load(response_model['Body'])
Top_pain_points_text= response_painpoints['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top painpoints:\n{Top_pain_points}\n")

In [ ]:
# Convert this into a function
Top_painpoints = Top_pain_points_text.replace("• ","").split(',')
Top_painpoints = [x.strip() for x in Top_painpoints]
print(f"Applicable painpoint labels found by LLMs:\n{Top_painpoints}")

#### 3.2) Finding the personalized offerings using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Offerings Document ***
{offering}
*** Offerings Document ***


Question: {customer['Description']} Their top-3 painpoints are {Top_pain_points}.

List and briefly advertise the top-3 offerings for {customer['Name']} based on the offerings document above.
Provide your list in bullet points without any preamble (e.g. • an offering: short description, • an offering: short description, ...). 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
params = {"max_new_tokens": 384, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_offerings = json.load(response_model['Body'])
Top_offerings_text= response_offerings['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top offerings:\n{Top_offerings_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'

matches = re.findall(pattern, Top_offerings_text)
Top_offerings = [] #(label, description)
for match in matches:
    title = match[0]
    description = match[1]
    Top_offerings.append((title,description))

In [ ]:
print(f"Applicable offerings found and their advertisements created by LLMs:\n")
[print(f"{key}: {value}\n")for (key, value) in Top_offerings]

#### 3.3) Art Design

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a UI/UX designer personalizing visual website content such as hero image and icons tailored to customer profile, personalized offerings and pain-points. 
Your task is to describe a hero image and six icons. For the Hero Image, design an attention-grabbing image that relates to the customer's industry, location, and mission. For icons, design simple, memorable icons that visually represent the concepts.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

For the Hero Image:
- Describe a scene that represents the customer's industry and mission
- Include elements that reflect the customer location
- Mention 1-2 key visual elements

For each icon:
- Describe a simple, easily recognizable shape or symbol
- Mention one key visual element
- Call our that this is an 'icon'.

Each icon should be distinct yet part of a cohesive set in style and color.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide your response in the following structured format:

** Hero Image[Description in 1-2 sentences]**
** {Top_painpoints[0]} Icon [Description in one sentence]**
** {Top_painpoints[1]} Icon [Description in one sentence]**
** {Top_painpoints[2]} Icon [Description in one sentence]**
** {Top_offerings[0][0]} Icon [Description in one sentence]**
** {Top_offerings[1][0]} Icon [Description in one sentence]**
** {Top_offerings[2][0]} Icon [Description in one sentence]**

Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_art= json.load(response_model['Body'])
art_descriptions_text = response_art['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**,','**')
print(f"Raw Output - top offerings:\n{art_descriptions_text}\n")

In [ ]:
# Creating the variable for the art descriptions that will help us to create a better prompt later
art_descriptions = art_descriptions_text.split("**")[1:]
art_descriptions = {art_descriptions[i].strip():{"Description":art_descriptions[i+1].strip(), \
                    'FileName':art_descriptions[i].replace(' ','')+'.jpg'} for i in range(0,int(len(art_descriptions)),2)}

In [ ]:
print(f"Here is the art and their descriptions:\n {json.dumps(art_descriptions, indent=4)}")

#### 5) Image Creation based on Description


##### 5.1) Creating the imagery from the descriptions
Notice that if the image is "icon" then 512 x 512 otherwise 1024 x 1024

In [ ]:
%%time
responses_image = []
images = []
image_files = []
for art, content in art_descriptions.items():
    if 'Icon' in art:
        art_height = 512
        art_width = 512
    else:
        art_height = 1024
        art_width = 1024
        

    response = Bedrock.invoke_stable_diff(prompt = content['Description'],  seed=0, height = art_height, 
                                          width = art_width)
    responses_image.append(response)
    image_bytes = base64.b64decode(response.encode('ascii'))
    image_files.append(content['FileName'])

    # Save the image
    image = Image.open(io.BytesIO(image_bytes))
    image.save(f"{content['FileName']}")
    images.append(np.array(image))

In [ ]:
print("Number of visual assets created: {}".format(len(image_files)))

##### b) Invoke Stable Diffusion
Here we assumed that the image description is the prompt for Text-to-Image. We successively call the APIs and then save the files.

In [ ]:
shutil.copy("./references/anycompany_logo.jpg", "./anycompany_logo.jpg")

Let's plot the images generated

In [ ]:
# Plot the images
fig, axs = plt.subplots(1, len(images), figsize=(30, 20))

for i, (image, ax, filename) in enumerate(zip(images, axs, image_files)):
    ax.imshow(image)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

#### 6) Let's Generate our HTML file

First, we need to extract the response for the Section 1: Detailed Description from *response_personalized_website*

In [ ]:
# Extract Section-1 using regular expressions
#section_1_pattern = r"Section 1: Detailed Website Description\n\n(.*?)\n\nSection 2:"
#match = re.search(section_1_pattern, response_personalized_website, re.DOTALL)

#if match:
#    response_detailed_explanation = match.group(1)
#    print(f"Here is the Section-1 extracted from the LLM response:\n\n{response_detailed_explanation}")
#else:
#    print("Section-1 not found in the input text.")

Here we will use Haiku to create the HTML file.

In [ ]:
prompt = f"""
You are an experienced front-end web developer specializing in creating accessible, responsive, and visually appealing websites. Your task is to generate the complete HTML, CSS, and JavaScript code that accurately implements the provided 'Website Description' while adhering to the specified guidelines.

<website description>
Know that, this your Design Guideline (Requirements):
{design_guideline}

You use the testimonials as follows;
{testimonials}

Website Description:
{response_personalized_website}
</website description>

Please carefully read the 'Website Description' line by line, and then generate the HTML, CSS, and JavaScript code required to build the described website while following the specified design guidelines and requirements.

Provide the HTML, CSS, and JavaScript code directly, starting with the <!DOCTYPE html> declaration, without any preamble or introduction.

"""

In [ ]:
%%time
response_html = Bedrock.invoke(prompt=prompt, modelID=model_id['Haiku'], max_tokens = 4096, temp = 0)

In [ ]:
#print(response_html)

#### Putting the assets into a single folder

In [ ]:
#Creating the folder for our project
try:
    os.mkdir(project)
    print(f"Project Folder: {project}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from IPython.display import HTML

In [ ]:
with open('main.html', 'w', encoding='utf-8') as file:
    file.write(response_html)

In [ ]:
project_files = list(visual_assets.keys())
project_files.append('main.html')

In [ ]:
for file in project_files:
    destination = project + "/" + file
    shutil.move(file, destination)